##### usage of yolo for getting labels only

##### lib and function def

In [5]:
import cv2
import torch
from PIL import Image
import cv2
import torch
from PIL import Image
import glob
import matplotlib.pyplot as plt

import numpy as np
import matplotlib.image as mpimg
import os
from pyzbar.pyzbar import decode
from pyzbar import pyzbar
import cv2
import glob
from tqdm import tqdm
from barcode import EAN13
from barcode.writer import ImageWriter
import matplotlib.pyplot as plt

import cv2
import torch
from PIL import Image


%matplotlib inline

## Objects:
#1. Label
class Label:
    def __init__(self,  boundPoints = None, labelImage = None, originCoordinate = None):
        self.boundPoints = boundPoints
        self.labelImage = labelImage
        self.originCoordinate = originCoordinate
        self.barcodeReadStatus = False
    
        #magnified 
        self.magnifiedImage = None
        self.magnificationFactor = 1
        
        #binarized image
        self.binarizedImage = None

        #decoded barcode
        self.barcodeList = []
        
        #bounding box
        self.boundingBox = None
        

#### files and model loading

In [6]:
files = glob.glob('./Dataset_for_training//*.png')
print(len(files))
another_files = glob.glob('./Dataset_for_training/*.jpg')

for file in another_files:
    files.append(file)
    
    
print(len(files))

0
107


In [7]:
#Model loading - detect locations of labels
model = torch.hub.load('ultralytics/yolov5', 'custom', path='./best.pt')
model.iou = 0.1
print(type(model))

Using cache found in C:\Users\sendr/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2021-12-9 torch 1.8.2+cu111 CUDA:0 (GeForce RTX 3050 Laptop GPU, 4096MiB)

Fusing layers... 
Model Summary: 224 layers, 7053910 parameters, 0 gradients
Adding AutoShape... 


<class 'models.common.AutoShape'>


In [14]:
class ImageFile:
    def __init__(self, image = None):
        self.image = image
        self.results = []
        self.stickers = []

In [9]:
'''
plt.imshow(cv2.imread(files[0])[:,:,::-1])


img2 = cv2.imread(files[0])
#img2.shape
test_image = img2[:,:,::-1]
plt.imshow(img2)
plt.show()
plt.imshow(test_image)
plt.show()

img2 = cv2.imread(files[0])  # OpenCV image (BGR tofiles[0] RGB)'''



imgs = []  # batch of images

for file in files:
    imgs.append(ImageFile(cv2.imread(file)))

for i in range (len(imgs)):
    imgs[i].image = cv2.cvtColor(imgs[i].image, cv2.COLOR_BGR2RGB)
    
for i in range (len(imgs)):
    imgs[i].results = model(np.copy(imgs[i].image))
   
for i in range (len(imgs)):
    imgs[i].results.save()
    df = imgs[i].results.pandas().xyxy[0]
    
    if (len(df) >0):
        for j in range (len(df)):
            xmin = int(df.iloc[j].xmin)
            ymin = int(df.iloc[j].ymin)
            xmax = int(df.iloc[j].xmax)
            ymax = int(df.iloc[j].ymax)
            imgs[i].stickers.append(imgs[i].image[ymin:ymax,xmin:xmax])
    
#Collect all stickers from all images (our sole purpose)
sticker_list = []

for img in imgs:
    for sticker in img.stickers:
        sticker_list.append(sticker)

#check before output  
print(len(sticker_list))
print(type(sticker_list[0]))

Saved 1 image to runs\detect\exp
Saved 1 image to runs\detect\exp2
Saved 1 image to runs\detect\exp3
Saved 1 image to runs\detect\exp4
Saved 1 image to runs\detect\exp5
Saved 1 image to runs\detect\exp6
Saved 1 image to runs\detect\exp7
Saved 1 image to runs\detect\exp8
Saved 1 image to runs\detect\exp9
Saved 1 image to runs\detect\exp10
Saved 1 image to runs\detect\exp11
Saved 1 image to runs\detect\exp12
Saved 1 image to runs\detect\exp13
Saved 1 image to runs\detect\exp14
Saved 1 image to runs\detect\exp15
Saved 1 image to runs\detect\exp16
Saved 1 image to runs\detect\exp17
Saved 1 image to runs\detect\exp18
Saved 1 image to runs\detect\exp19
Saved 1 image to runs\detect\exp20
Saved 1 image to runs\detect\exp21
Saved 1 image to runs\detect\exp22
Saved 1 image to runs\detect\exp23
Saved 1 image to runs\detect\exp24
Saved 1 image to runs\detect\exp25
Saved 1 image to runs\detect\exp26
Saved 1 image to runs\detect\exp27
Saved 1 image to runs\detect\exp28
Saved 1 image to runs\detect\e

418
<class 'numpy.ndarray'>


In [10]:
pwd

'D:\\Experiments'

In [11]:
# convert to RGB
for i in range (len(sticker_list)):
    try:
        #sticker[i] = cv2.cvtColor(sticker[i], cv2.COLOR_BGR2RGB)
        sticker[i] = sticker[i][:,:,::-1]
    except:
        continue
    
#outputting result
i = 0
prefix = ''
for sticker in tqdm(sticker_list):
    
    if (i < 10):
        prefix = '00'
    elif (i < 100):
        prefix = '0'
    elif (i>100):
        prefix = ''
        
    cv2.imwrite('./Stickers/sticker_'+prefix+str(i)+'.jpg',sticker[:,:,::-1])
    i+=1

100%|████████████████████████████████████████████████████████████████████████████████| 418/418 [00:09<00:00, 43.40it/s]


%matplotlib inline
plt.imshow(sticker_list[0][:,:,::-1])
plt.show()

###### another yolos

In [51]:
#Model loading - detect locations of labels
model2 = torch.hub.load('ultralytics/yolov5', 'custom', path='./best_sticker_OCR_All.pt')
model.iou = 0.1
print(type(model))

Using cache found in C:\Users\sendr/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2021-12-9 torch 1.8.2+cu111 CUDA:0 (GeForce RTX 3050 Laptop GPU, 4096MiB)

Fusing layers... 
Model Summary: 224 layers, 7062001 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


<class 'models.common.AutoShape'>


In [52]:
files = glob.glob('./Stickers/*.png')
print(len(files))
another_files = glob.glob('./Stickers/*.jpg')

for file in another_files:
    files.append(file)
    
    
print(len(files))

0
418


In [37]:
i = 0
for item in imgs[0].stickers:
    cv2.imwrite('sticker_'+str(i)+'.jpg',item)
    i+=1

In [38]:
import torch

# Model
#model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # or yolov5n - yolov5x6, custom

# Images
img = imgs[0].image  # or file, Path, PIL, OpenCV, numpy, list

# Inference
results = model2(img)

# Results
results.print()  # or .show(), .save(), .crop(), .pandas(), etc.

image 1/1: 711x1014 9 attributess, 8 datas
Speed: 20.0ms pre-process, 12.0ms inference, 1.0ms NMS per image at shape (1, 3, 480, 640)


In [41]:
results.show()

In [48]:
results.save('./Detection_Results/')

Saved 1 image to Detection_Results


In [54]:
result_list = []

for img in tqdm(imgs):
    result_list.append(model2(img.image))
    
i = 0
for result in tqdm(result_list):
    result.save('./Detection_Results/Img'+str(i))
    i+=1

  0%|                                                                                          | 0/418 [00:00<?, ?it/s]Saved 1 image to Detection_Results\Img0
Saved 1 image to Detection_Results\Img1
Saved 1 image to Detection_Results\Img2
Saved 1 image to Detection_Results\Img3
Saved 1 image to Detection_Results\Img4
Saved 1 image to Detection_Results\Img5
  1%|█▏                                                                                | 6/418 [00:00<00:07, 54.80it/s]Saved 1 image to Detection_Results\Img6
Saved 1 image to Detection_Results\Img7
Saved 1 image to Detection_Results\Img8
Saved 1 image to Detection_Results\Img9
Saved 1 image to Detection_Results\Img10
Saved 1 image to Detection_Results\Img11
Saved 1 image to Detection_Results\Img12
Saved 1 image to Detection_Results\Img13
Saved 1 image to Detection_Results\Img14
Saved 1 image to Detection_Results\Img15
Saved 1 image to Detection_Results\Img16
Saved 1 image to Detection_Results\Img17
Saved 1 image to Detection_Results

Saved 1 image to Detection_Results\Img138
Saved 1 image to Detection_Results\Img139
Saved 1 image to Detection_Results\Img140
Saved 1 image to Detection_Results\Img141
Saved 1 image to Detection_Results\Img142
 34%|███████████████████████████                                                    | 143/418 [00:01<00:02, 109.73it/s]Saved 1 image to Detection_Results\Img143
Saved 1 image to Detection_Results\Img144
Saved 1 image to Detection_Results\Img145
Saved 1 image to Detection_Results\Img146
Saved 1 image to Detection_Results\Img147
Saved 1 image to Detection_Results\Img148
Saved 1 image to Detection_Results\Img149
Saved 1 image to Detection_Results\Img150
Saved 1 image to Detection_Results\Img151
Saved 1 image to Detection_Results\Img152
Saved 1 image to Detection_Results\Img153
Saved 1 image to Detection_Results\Img154
 37%|█████████████████████████████▋                                                  | 155/418 [00:01<00:02, 96.02it/s]Saved 1 image to Detection_Results\Img155
Saved 

Saved 1 image to Detection_Results\Img271
Saved 1 image to Detection_Results\Img272
Saved 1 image to Detection_Results\Img273
Saved 1 image to Detection_Results\Img274
Saved 1 image to Detection_Results\Img275
 66%|████████████████████████████████████████████████████▊                           | 276/418 [00:02<00:01, 80.73it/s]Saved 1 image to Detection_Results\Img276
Saved 1 image to Detection_Results\Img277
Saved 1 image to Detection_Results\Img278
Saved 1 image to Detection_Results\Img279
Saved 1 image to Detection_Results\Img280
Saved 1 image to Detection_Results\Img281
Saved 1 image to Detection_Results\Img282
Saved 1 image to Detection_Results\Img283
Saved 1 image to Detection_Results\Img284
 68%|██████████████████████████████████████████████████████▌                         | 285/418 [00:03<00:01, 82.90it/s]Saved 1 image to Detection_Results\Img285
Saved 1 image to Detection_Results\Img286
Saved 1 image to Detection_Results\Img287
Saved 1 image to Detection_Results\Img288
Saved 

Saved 1 image to Detection_Results\Img412
Saved 1 image to Detection_Results\Img413
 99%|██████████████████████████████████████████████████████████████████████████████▏| 414/418 [00:03<00:00, 143.28it/s]Saved 1 image to Detection_Results\Img414
Saved 1 image to Detection_Results\Img415
Saved 1 image to Detection_Results\Img416
Saved 1 image to Detection_Results\Img417
100%|███████████████████████████████████████████████████████████████████████████████| 418/418 [00:04<00:00, 102.69it/s]


In [66]:
#folderz = os.listdir('./Detection_Results/')
folders = glob.glob('./Detection_Results/Img332/*.jpg')
print(type(folders))
#print(len(folderz))

<class 'list'>


In [86]:
pwd

'D:\\Experiments'

In [92]:
image_result_list = []

for folder in tqdm(os.listdir('./Detection_Results/')):
    temp_list = glob.glob('./Detection_Results/'+folder+'/*.jpg')
    image_result_list.append(cv2.imread(temp_list[0]))
    
i = 0
for item in tqdm(image_result_list):
    cv2.imwrite('result_'+str(i)+'.jpg', item)
    i+=1

100%|████████████████████████████████████████████████████████████████████████████████| 418/418 [00:04<00:00, 97.62it/s]
